# (Tang et al., 2010) Overlapping Experiment Infrastructure: More, Better, Faster Experimentation

link: https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/36500.pdf

## Abstract

This paper describes Google's overlapping experiment infrastructure that is a key component to solving the problems of scaling experimentation.

## 1. Introduction

Limiting the rate of change by the number of simultaneous experiments we can run is simply not acceptable.

The design goals for our experiment infrastructure are therefore:
- **More**: Need scalability to run more experiments simultaneously, but also need flexibility.
- **Better**: Invalid experiments should not be allowed to run on live traffic. Standardized metrics should be easily available for all experiments so that experiment comparisons are fair.
- **Faster**: Should be easy and quick to set up an experiment

For an experiment infrastructure, the obvious solutions are either to have a single layer of experiments or to have multi-factorial experiment design.

**Single Layer**: Every query is in at most one experiment. Easy and flexible, but insufficiently scalable.

**Multi-factorial Experimental Design**: common in statistical literature, where each parameter can be experimented on independently; each experimental value for a parameter overlaps with every other experiment value for all of the other parameters. Effectively each query would be in $N$ experiments simultaneously, where $N$ equals the # of parameters. While backed by years of research and practice, when there are thousands of parameters that cannot necessarily be varied independently, this can be difficult. An example is a two parameter system, one for the background color of a web page and another for the text color. While "blue" may be a valid value for both, if both parameters are blue at the same time, then the page will be unreadable.

The solution in this paper is to partition the parameters into subsets, and each subset contains parameters that cannot be varied independently of each other.

A subset is associated with a layer that contains experiments, and traffic diversion into experiments in different layers are *orthogonal*. Each query here would be in $N$ experiments, where $N$ equals the # of layers.

## 2. Related Work

Work related to experimentation falls roughly in 3 areas:
1. Copious statistical literature on multi-factorial experiments
2. How to run web experiments
3. Interleaved experiments, which is focused on a specific type of experiment design used for evaluating ranking changes.

## 3. Background

Explains at a high level how google traffic and search works. For instance, they explain how they use cookies as the basis of diversion (more specificallly a cookie mod, so for example, given a numeric representation of a cookie, take that number modulo 1000, and all cookies with the same mod value would be grouped together for experiment diversion)

Prior to developing our overlpaping experiment infrastructure, used a basic single layer infrastructure. In such an infrastructure, each query is in at most one experiment. Cookie-mod-base experiments were diverted first, followed by random-traffic based experiments. Upstream binaries got "first dibs" on a query, and if the upstream binaries were running enough experiments, then downstream binaries could be starved for traffic to run experiments on. This was easy to use and reasonably flexible, however was not scalable.

## 4. Overlapping Experiment Infrastructure

Tries to keep the advantages of a single layer system (ease of use, speed) while increasing scalability, flexibility, and robustness.

Unfortunately, a fully multi-factorial experiment does not work as not all parameters that are randomized want to be paired together (e.g., pink background with a pink text)

Given this constraint, the main idea is to partition parameters into $N$ subsets. Each subset is associated with a layer of experiments. Each request would be in at most $N$ experiments simultaneously (one experiment per layer). Each experiment can only modify parameters associated with its layer (subset), and the same parameter cannot be associated with multiple layers.

So how do you partition the parameters? First, parameters from different binaries can be in different subsets. However, all parameters for a given binary do not need to be in a single subset: we can further partition the parameters within a binary either by examination or by examining past experiments.



## 5. Tools & Processes

While having the overlapping infrastructure is necessary to be able to scale to running more experiments faster and evaluate more changes concurrently, the infrastructure by itself is not sufficient. We also need tools, research, and educational processes to support the faster rate of experimentation.

### 5.1 Tools

Data File Checks

Real-time Monitoring

### 5.2 Experiment Design & Sizing

**5.3.2 Sizing**:

As Kohavi mentions, experiments should be sized to have enough statistical power to detect as small a metric change as considered interesting or actionable. here we discuss how to size experiments and the dependency on experiment set-up as well as an associated experiment sizing tool.

Define the effective size of an experiment as:

$$N=(1/\text{queries}_{\text{control}}+1/\text{queries}_{\text{experiment}})^{-1}$$

In practice we are interested in the individual terms queries for control and treatment, but it is through $N$ that these affect the variance of the relative metric estimates. To determine $N$ correctly, we need to know:
- Which metric(s) the experimenter cares about
- For each metric, what change the experimenter would like to detect ($\theta$), e.g., the experimenter wants to be able to detect a 2% change in the CTR
- For each metric, the standard error for a one unit (i.e., $N=1$) sample (s). Thus the standard error for an experiment of size $N$ is $s/\sqrt{N}$.

Kohavi assumes that the experiment and control are the same size, and so the sample size must be greater than or equal to $16(s/\theta)^2$ to meet the detection requirement. The number 16 is determined both by the desired confidence level ($1-\alpha$, often 95%) and esired statistical power ($1-\beta$, often 80%)

An advantage of the overlapping set-up is that we can create a large control in each layer that can be shared among multiple experiments. If the shared control is much larger than the experiment, we can have smaller experiment size while gaining statistical power.

### 5.3 Fast Analytics

Even if the infrastructure and tools are sped up, an experiment will not be fully expedited if the experiment analysis process is not addressed. The primary goal of the analysis tool is to provide accurate values for the suite of metrics that experimenters examine to evaluate their experiment.

Beyond accuracy and completeness, other key design goals for an experiment analysis tool include:
- Correctly computed and displayed confidence intervals
- A good UI
- Support for Slicing
- Extensibilty

### 5.4 Education

Need to address the people-side. Two processes have helped to ensure that experiments are well-designed and that the result of an experiment are understood and disseminated.

**5.4.1 Experiment Council**:

a group of engineers who review a light-weight checklist that experimenters fill out prior to running their experiment.

**5.4.2 Interpreting the Data**:

A forum where experimenters bring their experiment results to discuss with experts